<a href="https://colab.research.google.com/github/innovativenexusbd/AgroAI/blob/main/nutrientdataprediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing

In [ ]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report


Load the Dataset and Checking the overall sufficiency

In [ ]:
df = pd.read_csv('soil_nutrients.csv')

THRESHOLDS = {
    'Nitrogen': 30,  # mg/kg
    'Phosphorus': 15,  # mg/kg
    'Potassium': 20  # mg/kg
}
def check_sufficiency(row, nutrient, threshold):
    return 'Sufficient' if row[nutrient] >= threshold else 'Insufficient'
for nutrient, threshold in THRESHOLDS.items():
    df[nutrient + '_Status'] = df.apply(lambda row: check_sufficiency(row, nutrient, threshold), axis=1)
def overall_sufficiency(row):
    if (row['Nitrogen_Status'] == 'Sufficient' and
        row['Phosphorus_Status'] == 'Sufficient' and
        row['Potassium_Status'] == 'Sufficient'):
        return 'Sufficient'
    else:
        return 'Insufficient'
df['Overall_Status'] = df.apply(overall_sufficiency, axis=1)


Prepare Data and Split the Dataset

In [ ]:
X = df[['Nitrogen', 'Phosphorus', 'Potassium']]
y = df['Overall_Status']
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.3, random_state=42)

Initiating and Training the KNN Model

In [ ]:
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

Prediction

In [ ]:
y_pred = knn.predict(X_test)
y_pred_decoded = label_encoder.inverse_transform(y_pred)
y_test_decoded = label_encoder.inverse_transform(y_test)
print(classification_report(y_test_decoded, y_pred_decoded))

              precision    recall  f1-score   support

Insufficient       0.00      0.00      0.00         1
  Sufficient       0.50      1.00      0.67         1

    accuracy                           0.50         2
   macro avg       0.25      0.50      0.33         2
weighted avg       0.25      0.50      0.33         2



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Saving the Ultimate Result in the CSV format

In [ ]:
predictions_df = X_test.copy()
predictions_df['Predicted_Status'] = y_pred_decoded
predictions_df.to_csv('predicted_soil_nutrient_status.csv', index=False)